In [97]:
import pandas as pd
from  datetime import datetime, timedelta
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, monotonically_increasing_id, udf, to_date
from pyspark.sql.types import (StructType,
                               StructField,
                               StringType,
                               IntegerType,
                               DoubleType,
                               DateType,
                               FloatType)

In [2]:
spark = SparkSession.builder.\
    appName("data_spark_on_emr").\
    config("spark.jars.packages","saurfang:spark-sas7bdat:3.0.0-s_2.12").\
    getOrCreate()

22/05/27 17:15:01 WARN Utils: Your hostname, OneForAll-NickdeMacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.0.0.101 instead (on interface en0)
22/05/27 17:15:01 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/usr/local/Cellar/apache-spark/3.2.1/libexec/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/oneforall_nick/.ivy2/cache
The jars for the packages stored in: /Users/oneforall_nick/.ivy2/jars
saurfang#spark-sas7bdat added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-7b149d6f-f5c6-43b0-a912-d7498a47fab5;1.0
	confs: [default]
	found saurfang#spark-sas7bdat;3.0.0-s_2.12 in spark-packages
	found com.epam#parso;2.0.11 in central
	found org.slf4j#slf4j-api;1.7.5 in central
	found org.apache.logging.log4j#log4j-api-scala_2.12;12.0 in central
	found org.scala-lang#scala-reflect;2.12.10 in central
	found org.apache.logging.log4j#log4j-api;2.13.2 in central
:: resolution report :: resolve 333ms :: artifacts dl 12ms
	:: modules in use:
	com.epam#parso;2.0.11 from central in [default]
	org.apache.logging.log4j#log4j-api;2.13.2 from central in [default]
	org.apache.logging.log4j#log4j-api-scala_2.12;12.0 from central in [default]
	org.scala-lang#scala-reflect;2.12.10 from central in [default]
	org.slf4j#slf4j-api;1.7.5 fro

In [42]:
spark

In [4]:
# Stop spark session if I don't need it.
# spark.stop()

In [4]:
# spark session setting configuration
spark.conf.set("spark.sql.shuffle.partitions", "5")
spark.sparkContext.getConf().getAll()

[('spark.sql.parquet.binaryAsString', 'true'),
 ('spark.driver.extraJavaOptions',
  '"-Dio.netty.tryReflectionSetAccessible=true"'),
 ('spark.driver.memory', '4g'),
 ('spark.executor.memory', '4g'),
 ('spark.driver.host', '10.0.0.101'),
 ('spark.app.startTime', '1653642903338'),
 ('spark.executor.id', 'driver'),
 ('spark.jars',
  'file:///Users/oneforall_nick/.ivy2/jars/saurfang_spark-sas7bdat-3.0.0-s_2.12.jar,file:///Users/oneforall_nick/.ivy2/jars/com.epam_parso-2.0.11.jar,file:///Users/oneforall_nick/.ivy2/jars/org.apache.logging.log4j_log4j-api-scala_2.12-12.0.jar,file:///Users/oneforall_nick/.ivy2/jars/org.slf4j_slf4j-api-1.7.5.jar,file:///Users/oneforall_nick/.ivy2/jars/org.scala-lang_scala-reflect-2.12.10.jar,file:///Users/oneforall_nick/.ivy2/jars/org.apache.logging.log4j_log4j-api-2.13.2.jar'),
 ('spark.driver.port', '49462'),
 ('spark.sql.warehouse.dir',
  'file:/Users/oneforall_nick/workspace/Udacity_capstone_project/aws_emr_steps/spark-warehouse'),
 ('spark.app.initial.file

In [ ]:
# READ ALL DATA AND SAVE TO simulation data file, and then analysis those data in spark.
# ****** Read data from local file system(be a s3) to spark DataFrame ******
# All data format will be saved as a parquet file.

# file path: data >> immigration_data
# There are three tables: immigration_table, immigration_personal_table and immigration_label_table
# ***immigration_table, immigration_personal_table and immigration_label_table schema***
""""Table: immigration_main_information schema
pk: cicid -> imm_main_cic_id
1. i94yr  -> imm_year
2. i94mon -> imm_month
3. i94citi&i94res -> imm_citi_res -> imm_cntyl
4. i94visa -> imm_visa
    three categories:
        1 = Business
        2 = Pleasure
        3 = Student
5. i94port -> imm_port
6. arrdate -> imm_arrival_date:
7. i94mode -> imm_model:
    four categories:
        1 = 'Air'
	    2 = 'Sea'
	    3 = 'Land'
	    9 = 'Not reported'
8. i94addr -> imm_address
    ex: 'AL'='ALABAMA'
9. airline -> imm_airline
10 fltno -> imm_flight_no
    schema: StringType()
"""


"""Table: immigration_personal schema -> According to this person data that I will make a core data table to display notifications information.
pk: cicid -> imm_per_cic_id
    schema: StringType()
1. biryear -> imm_person_birth_year
    schema: IntegerType()
2. gender -> imm_person_gender
    schema: StringType()
3. visatype -> imm_person_visa_type
    schema: StringType()
"""




# Dimension Table: immigration_label

- df_imm_city_res_label
- df_imm_destination_city
- df_imm_travel_code
- df_imm_address
- df_imm_visa

In [235]:
label_data = "/Users/oneforall_nick/workspace/Udacity_capstone_project/airflow/data/immigration_data/immigration_labels_descriptions.SAS"

# Get data column attributes
# from sas7bdat import SAS7BDAT

# Using the library for knowing more data information
# imm_label_data_read = SAS7BDAT(label_data)

# t = list(imm_label_data_read)
# TODO: make sure that is a correct data

# def read_data_to_df(call_func):
#     # def wrapper(label_data):
#     def wrapper(*args, **kwargs):
#         # return call_func(*args)
#         print("Reading data...")
#         with open(label_data) as f:
#             context = f.read().replace('\t', '')
#         call_func(context, *args, **kwargs)
#         print("Data reading finished.")
#     return wrapper


with open(label_data) as f:
    context = f.read().replace('\t', '')
    # content_mapping = context[context.index('i94prtl'):]
    # content_line_split = content_mapping[:content_mapping.index(
    #     ';')].split('\n')
    # content_line_list = [line.replace("'", "") for line in content_line_split]
    # content_dict = [i.split('=') for i in content_line_list[1:]]
    # # content_dict = [[i[0].strip(), i[1].strip(), [e for e in i[1].strip().split(', ')[1:]]] for i in content_dict if len(i) == 2]
    # content_dict = [[i[0].strip(), i[1].strip().split(', ')[:][0], e] for i in content_dict if len(i) == 2 for e in i[1].strip().split(', ')[1:]]


# @read_data_to_df
def code_mapping(context, idx):
    content_mapping = context[context.index(idx):]
    content_line_split = content_mapping[:content_mapping.index(';')].split('\n')
    content_line_list = [line.replace("'", "") for line in content_line_split]
    content_dict = [i.split('=') for i in content_line_list[1:]]
    content_dict = [[i[0].strip(), i[1].strip().split(', ')[:][0], e] for i in content_dict if len(i) == 2 for e in i[1].strip().split(', ')[1:]]
    return content_dict

# i94cit_res = code_mapping(context, "i94cntyl")
i94port = code_mapping(context, "i94prtl")
# i94mode = code_mapping(context, "i94model")
# i94addr = code_mapping(context, "i94addrl")
# i94visa = {'1': 'Business',
#            '2': 'Pleasure',
#            '3': 'Student'}

# for i in i94port:
#     print(i)

In [236]:
# df_imm_destination_city = spark.sparkContext.parallelize(i94port).toDF(["code_of_imm_destination_city", "value_of_imm_destination_city", "code1"])

In [240]:
# TODO: Writing into aws s3 and format parquet file.
df_imm_city_res_label = spark.sparkContext.parallelize(i94cit_res.items()).toDF(["col_of_imm_cntyl", "value_of_imm_cntyl"])\
    .withColumn("col_of_imm_cntyl", col("col_of_imm_cntyl").cast("Integer"))\
    .withColumn("value_of_imm_cntyl", col("value_of_imm_cntyl").cast("String"))

df_imm_destination_city = spark.sparkContext.parallelize(i94port).toDF(["code_of_imm_destination_city", "value_of_imm_destination_city", "value_of_alias_imm_destination_city"])\
    .withColumn("code_of_imm_destination_city", col("code_of_imm_destination_city").cast("String")) \
    .withColumn("value_of_imm_destination_city", col("value_of_imm_destination_city").cast("String")) \
    .withColumn("value_of_alias_imm_destination_city", col("value_of_alias_imm_destination_city").cast("String"))

df_imm_destination_city_tmp = df_imm_destination_city.createOrReplaceTempView("imm_destination_city_data")
df_imm_destination_city_tmp = spark.sql("SELECT * FROM imm_destination_city_data")

# df_imm_destination_city_tmp = df_imm_destination_city.createOrReplaceTempView("imm_destination_city")
# df_imm_destination_city_tmp = spark.sql("SELECT * FROM imm_destination_city WHERE code_of_imm_destination_city IS NOT NULL")

df_imm_travel_code = spark.sparkContext.parallelize(i94mode.items()).toDF(["code_of_imm_travel_code", "value_of_imm_travel_code"])\
    .withColumn("code_of_imm_travel_code", col("code_of_imm_travel_code").cast("Integer"))\
    .withColumn("value_of_imm_travel_code", col("value_of_imm_travel_code").cast("String"))

df_imm_address = spark.sparkContext.parallelize(i94addr.items()).toDF(["code_of_imm_address", "value_of_imm_address"])\
    .withColumn("code_of_imm_address", col("code_of_imm_address").cast("String"))\
    .withColumn("value_of_imm_address", col("value_of_imm_address").cast("String"))

df_imm_visa = spark.sparkContext.parallelize(i94visa.items()).toDF(["code_of_imm_visa", "value_of_imm_visa"])\
    .withColumn("code_of_imm_visa", col("code_of_imm_visa").cast("Integer"))\
    .withColumn("value_of_imm_visa", col("value_of_imm_visa").cast("String"))



# df_imm_city_res_label.createOrReplaceTempView("imm_city_res_label")

# df_imm_city_res_label = spark.sql("SELECT * FROM imm_city_res_label")
# df_imm_city_res.persist()

In [248]:
df_imm_destination_city_tmp.show(n=5, truncate=10)


+----------------------------+-----------------------------+-----------------------------------+
|code_of_imm_destination_city|value_of_imm_destination_city|value_of_alias_imm_destination_city|
+----------------------------+-----------------------------+-----------------------------------+
|                         ALC|                        ALCAN|                                 AK|
|                         ANC|                    ANCHORAGE|                                 AK|
|                         BAR|                   BAKER A...|                                 AK|
|                         DAC|                   DALTONS...|                                 AK|
|                         PIZ|                   DEW STA...|                                 AK|
+----------------------------+-----------------------------+-----------------------------------+
only showing top 5 rows



In [32]:
# TODO: Check data source
for column in df_imm_city_res_label.columns:
    n = df_imm_city_res_label.select(column).distinct().count()
    print(n)

289
287


In [34]:
df_imm_city_res.unpersist()

DataFrame[col_of_imm_cntyl: int, value_of_imm_cntyl: string]

# Dimension Table: news_article


In [108]:
# file path: data >> news_article
"""Table: news_article schema
pk: cord_uid -> news_cord_uid
1. source_x -> news_source
    schema: StringType()
2. title -> news_title
    schema: StringType()
3. license -> news_licence
    schema: StringType()
4. abstract -> news_abstract
    schema: StringType()
5. publish_time -> news_publish_time (fk)
    schema: TimestampType()
6. authors -> news_authors
    schema: StringType()
7. url -> news_url
    schema: StringType()
"""
data_news = "/Users/oneforall_nick/workspace/Udacity_capstone_project/airflow/data/news_data/metadata.csv"

# news_schema = StructType([
#     StructField(name="news_cord_uid", dataType=StringType(), nullable=True),
#     StructField(name="news_source", dataType=StringType(), nullable=True),
#     StructField(name="news_title", dataType=StringType(), nullable=True),
#     StructField(name="news_licence", dataType=StringType(), nullable=True),
#     StructField(name="news_abstract", dataType=StringType(), nullable=True),
#     StructField(name="news_publish_time", dataType=DateType(), nullable=True),
#     StructField(name="news_authors", dataType=StringType(), nullable=True),
#     StructField(name="news_url", dataType=StringType(), nullable=True)
# ])

df_news = spark.read.options(header=True, delimiter=',').csv(path=data_news)

df_news = df_news.withColumn("news_cord_uid", col("cord_uid").cast("String")) \
                    .withColumn("news_source", col("source_x").cast("String")) \
                        .withColumn("news_title", col("title").cast("String")) \
                            .withColumn("news_licence", col("license").cast("String")) \
                                .withColumn("news_abstract", col("abstract").cast("String")) \
                                    .withColumn("news_publish_time", to_date(df_news.publish_time, "yyyy-MM-dd")) \
                                        .withColumn("news_authors", col("authors").cast("String")) \
                                            .withColumn("news_url", col("url").cast("String")) \
                 .select(col("news_cord_uid"),
                         col("news_source"),
                         col("news_title"),
                         col("news_licence"),
                         col("news_abstract"),
                         col("news_publish_time"),
                         col("news_authors"),
                         col("news_url")
                         )

df_news_tmp = df_news.createOrReplaceTempView("news_article_data")

df_news_tmp = spark.sql("SELECT DISTINCT publish_time FROM news_article_data")

df_news_tmp.persist()

df_news_tmp.explain()


== Physical Plan ==
InMemoryTableScan [publish_time#6445]
   +- InMemoryRelation [publish_time#6445], StorageLevel(disk, memory, deserialized, 1 replicas)
         +- *(2) HashAggregate(keys=[publish_time#6445], functions=[])
            +- Exchange hashpartitioning(publish_time#6445, 5), ENSURE_REQUIREMENTS, [id=#1786]
               +- *(1) HashAggregate(keys=[publish_time#6445], functions=[])
                  +- FileScan csv [publish_time#6445] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/oneforall_nick/workspace/Udacity_capstone_project/airflow/..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<publish_time:string>




In [92]:
df_news_tmp.unpersist()


AttributeError: 'NoneType' object has no attribute 'unpersist'

# Dimension Table: Us Cities Demographics data

In [245]:
# Create a us-cities data dimension table
"""Table: us_cities_demographics schema
pk: generated -> cidemo_id
    schema: IntegerType()
1. City -> cidemo_city
    schema: StringType()
2. State -> cidemo_state
    schema: StringType()
3. Median Age -> cidemo_median_age
    schema: FloatType()
4. Total Population -> cidemo_total_population
    schema: IntegerType()
5. State Code -> cidemo_state_code (fk)
    schema: StringType()
6. Count -> cidemo_count
    schema: IntegerType()
"""

data_us_cities_demographics = "/Users/oneforall_nick/workspace/Udacity_capstone_project/airflow/data/usCitiesDemographics_data/usCitiesDemo.csv"

# TODO -> Must be defined a function that generated each table schema:
us_cities_demographics_data_schema = StructType([
    StructField(name="cidemo_city", dataType=StringType(), nullable=True),
    StructField(name="cidemo_state", dataType=StringType(), nullable=True),
    StructField(name="cidemo_median_age", dataType=FloatType(), nullable=True),
    StructField(name="cidemo_total_population", dataType=IntegerType(), nullable=True),
    StructField(name="cidemo_state_code", dataType=StringType(), nullable=True),
    StructField(name="cidemo_count", dataType=IntegerType(), nullable=True)
])

# Using pyspark to read csv file
df_news = spark.read.options(header=True, delimiter=';').csv(data_us_cities_demographics)
"""
root
 |-- cidemo_city: string (nullable = true)
 |-- cidemo_state: string (nullable = true)
 |-- cidemo_median_age: float (nullable = true)
 |-- cidemo_total_population: integer (nullable = true)
 |-- cidemo_state_code: string (nullable = true)
 |-- cidemo_count: integer (nullable = true)
"""
df_news = df_news.withColumn("cidemo_city", col("City").cast("String")) \
                    .withColumn("cidemo_state", col("State").cast("String")) \
                        .withColumn("cidemo_median_age", col("Median Age").cast("Float")) \
                            .withColumn("cidemo_male_population", col("Male Population").cast("Integer")) \
                                .withColumn("cidemo_female_population", col("Female Population").cast("Integer")) \
                                    .withColumn("cidemo_total_population", col("Total Population").cast("Integer")) \
                                            .withColumn("cidemo_number_of_veterans", col("Number of Veterans").cast("Integer")) \
                                                .withColumn("cidemo_foreign_born", col("Foreign-born").cast("Integer")) \
                                                    .withColumn("cidemo_average_household_size", col("Average Household Size").cast("Float")) \
                                                        .withColumn("cidemo_state_code", col("State Code").cast("String")) \
                                                            .withColumn("cidemo_race", col("Race").cast("String")) \
    .withColumn("cidemo_count", col("Count").cast("Integer")) \
                    .select(col("cidemo_city"),
                            col("cidemo_state"),
                            col("cidemo_median_age"),
                            col("cidemo_total_population"),
                            col("cidemo_state_code"),
                            col("cidemo_count"))

# Auto-generated series of id
df_us_cities_demographics = df_news.withColumn("cidemo_id", monotonically_increasing_id())

df_us_cities_demographics_temp = df_news.createOrReplaceTempView("us_cities_demographics_data")

df_us_cities_demographics_temp = spark.sql("SELECT * FROM us_cities_demographics_data")

df_us_cities_demographics_temp.persist()

df_us_cities_demographics_temp.explain()

== Physical Plan ==
InMemoryTableScan [cidemo_city#11347, cidemo_state#11361, cidemo_median_age#11376, cidemo_total_population#11427, cidemo_state_code#11509, cidemo_count#11556]
   +- InMemoryRelation [cidemo_city#11347, cidemo_state#11361, cidemo_median_age#11376, cidemo_total_population#11427, cidemo_state_code#11509, cidemo_count#11556], StorageLevel(disk, memory, deserialized, 1 replicas)
         +- *(1) Project [City#11323 AS cidemo_city#11347, State#11324 AS cidemo_state#11361, cast(Median Age#11325 as float) AS cidemo_median_age#11376, cast(Total Population#11328 as int) AS cidemo_total_population#11427, State Code#11332 AS cidemo_state_code#11509, cast(Count#11334 as int) AS cidemo_count#11556]
            +- FileScan csv [City#11323,State#11324,Median Age#11325,Total Population#11328,State Code#11332,Count#11334] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/Users/oneforall_nick/workspace/Udacity_capstone_project/airflow/..., Partit

In [247]:
df_us_cities_demographics_temp.show(n=5, truncate=10)

+-----------+------------+-----------------+-----------------------+-----------------+------------+
|cidemo_city|cidemo_state|cidemo_median_age|cidemo_total_population|cidemo_state_code|cidemo_count|
+-----------+------------+-----------------+-----------------------+-----------------+------------+
| Silver ...|    Maryland|             33.8|                  82463|               MD|       25924|
|     Quincy|  Massach...|             41.0|                  93629|               MA|       58723|
|     Hoover|     Alabama|             38.5|                  84839|               AL|        4759|
| Rancho ...|  California|             34.5|                 175232|               CA|       24437|
|     Newark|  New Jersey|             34.6|                 281913|               NJ|       76402|
+-----------+------------+-----------------+-----------------------+-----------------+------------+
only showing top 5 rows



In [122]:
df_us_cities_demographics_temp.unpersist()

DataFrame[cidemo_city: string, cidemo_state: string, cidemo_median_age: float, cidemo_total_population: int, cidemo_state_code: string, cidemo_count: int]

# Dimension Table: Immigration data

In [69]:
# TODO: This code block is be placed in analysis jupyter notebook files
imm_data = "/Users/oneforall_nick/workspace/Udacity_capstone_project/airflow/data/immigration_data/immigration_apr16_sub.sas7bdat"


df_imm_data = spark.read.format("com.github.saurfang.sas.spark").load(imm_data)


In [70]:
type(df_imm_data)

pyspark.sql.dataframe.DataFrame

# Dimension Table: Immigration personal data

In [72]:
"""Table: immigration_personal schema -> According to this person data that I will make a core data table to display notifications information.
pk: cicid -> imm_per_cic_id
    schema: StringType()
1. biryear -> imm_person_birth_year
    schema: IntegerType()
2. gender -> imm_person_gender
    schema: StringType()
3. visatype -> imm_person_visa_type
    schema: StringType()
"""
# show(n=5, truncate=5)
df_immigration_personal = df_imm_data.withColumn("imm_per_cic_id", col("cicid").cast("String"))\
           .withColumn("imm_person_birth_year", col("biryear").cast("Integer"))\
           .withColumn("imm_person_gender", col("gender").cast("String"))\
           .withColumn("imm_visatype", col("visatype").cast("String")).select(col("imm_per_cic_id"), \
                                                                              col("imm_person_birth_year"), \
                                                                              col("imm_person_gender"), \
                                                                              col("imm_visatype"))

df_immigration_personal_tmp = df_immigration_personal.createOrReplaceTempView("imm_personal")

df_immigration_personal_tmp = spark.sql("SELECT * FROM imm_personal")

df_immigration_personal_tmp.persist()

df_immigration_personal_tmp.explain()

== Physical Plan ==
InMemoryTableScan [imm_per_cic_id#1564, imm_person_birth_year#1594, imm_person_gender#1625, imm_visatype#1657]
   +- InMemoryRelation [imm_per_cic_id#1564, imm_person_birth_year#1594, imm_person_gender#1625, imm_visatype#1657], StorageLevel(disk, memory, deserialized, 1 replicas)
         +- *(1) Project [cast(cicid#1333 as string) AS imm_per_cic_id#1564, cast(biryear#1353 as int) AS imm_person_birth_year#1594, gender#1355 AS imm_person_gender#1625, visatype#1360 AS imm_visatype#1657]
            +- *(1) Scan SasRelation(/Users/oneforall_nick/workspace/Udacity_capstone_project/airflow/data/immigration_data/immigration_apr16_sub.sas7bdat,null,false,false,false,None,false,false,false,false,60,None,None) [cicid#1333,i94yr#1334,i94mon#1335,i94cit#1336,i94res#1337,i94port#1338,arrdate#1339,i94mode#1340,i94addr#1341,depdate#1342,i94bir#1343,i94visa#1344,count#1345,dtadfile#1346,visapost#1347,occup#1348,entdepa#1349,entdepd#1350,entdepu#1351,matflag#1352,biryear#1353,dtadd

# Dimension Table: Immigration main data

In [126]:
""""Table: immigration_main_information schema
pk: cicid -> imm_main_cic_id
1. i94yr: 4 digit year of the arrival  -> imm_year
2. i94mon: numeric month of the arrival -> imm_month
3. i94citi&i94res: 3 digit code of origin city -> imm_citi_res -> imm_cntyl
4. i94visa: reason for immigration -> imm_visa
    three categories:
        1 = Business
        2 = Pleasure
        3 = Student
5. i94port: 3 character code of destination city --> Foreign key (used to map to USDemographics and City Temperature data) -> imm_port
6. arrdate: arrival date of the departure -> imm_arrival_date:
7. deptdate: departure date
date_add
7. i94mode: 1 digit travel code -> imm_model:
    four categories:
        1 = 'Air'
	    2 = 'Sea'
	    3 = 'Land'
	    9 = 'Not reported'
8. i94addr -> imm_address
    ex: 'AL'='ALABAMA'
9. airline -> imm_airline
10 fltno -> imm_flight_no
    schema: StringType()
"""


def convert_to_datetime(days: DoubleType) -> datetime:
    """convert_to_datetime converts days to datetime format

    Args:
        days (DoubleType): from sas arrive or departure date

    Returns:
        datetime: added days to datetime format result.
    """
    if days is not None:
        date = datetime.strptime('1960-01-01', '%Y-%m-%d')

        return date + timedelta(days=days)

udf_convert_to_datetime = udf(lambda x: convert_to_datetime(x), DateType())

# df_imm_data.show(n=5, truncate=5)
# t1 = df_imm_data.withColumn("imm_depdate", col("depdate")).select(col("imm_depdate")).printSchema()

immigration_main_information = df_imm_data.withColumn("imm_main_cic_id", col("cicid").cast("Integer"))\
            .withColumn("imm_year", col("i94yr").cast("Integer"))\
                .withColumn("imm_month", col("i94mon").cast("Integer"))\
                    .withColumn("imm_cntyl", col("i94cit").cast("Integer"))\
                        .withColumn("imm_visa", col("i94visa").cast("Integer"))\
                            .withColumn("imm_port", col("i94port").cast("String"))\
                                .withColumn("imm_arrival_date", udf_convert_to_datetime(col("arrdate")))\
                                    .withColumn("imm_departure_date", udf_convert_to_datetime(col("depdate")))\
                                        .withColumn("imm_model", col("i94mode").cast("Integer"))\
                                            .withColumn("imm_address", col("i94addr").cast("String"))\
                                                .withColumn("imm_airline", col("airline").cast("String"))\
                                                    .withColumn("imm_flight_no", col("fltno").cast("String"))\
        .select(col('imm_main_cic_id'), \
                    col('imm_year'),\
                        col('imm_month'),\
                            col('imm_cntyl'),\
                                col('imm_visa'),\
                                    col('imm_port'),\
                                        col('imm_arrival_date'),\
                                            col('imm_departure_date'),\
                                                col('imm_model'),\
                                                    col('imm_address'),\
                                                        col('imm_airline'),\
                                                            col('imm_flight_no'))

df_immigration_main_information = immigration_main_information.createOrReplaceTempView(
    "immigration_main_information_data")

df_immigration_main_information = spark.sql("SELECT * FROM immigration_main_information_data")

df_immigration_main_information.show(n=5, truncate=10)
# df_immigration_main_information.persist()

# df_immigration_main_information.explain()


+---------------+--------+---------+---------+--------+--------+----------------+------------------+---------+-----------+-----------+-------------+
|imm_main_cic_id|imm_year|imm_month|imm_cntyl|imm_visa|imm_port|imm_arrival_date|imm_departure_date|imm_model|imm_address|imm_airline|imm_flight_no|
+---------------+--------+---------+---------+--------+--------+----------------+------------------+---------+-----------+-----------+-------------+
|              6|    2016|        4|      692|       2|     XXX|      2016-04-29|              null|     null|       null|       null|         null|
|              7|    2016|        4|      254|       3|     ATL|      2016-04-07|              null|        1|         AL|       null|        00296|
|             15|    2016|        4|      101|       2|     WAS|      2016-04-01|        2016-08-25|        1|         MI|         OS|           93|
|             16|    2016|        4|      101|       2|     NYC|      2016-04-01|        2016-04-23|      

In [14]:
# Get data column attributes
# from sas7bdat import SAS7BDAT

# Using the library for knowing more data information
# imm_data_read = SAS7BDAT(imm_data)

# imm_data_read.header
# for i in imm_data_read.columns:
#     print("col_id ", i.col_id)
#     print("  name",  i.name.decode(encoding ='utf-8'))
#     print("  label", i.label.decode(encoding ='utf-8'))
#     print("  format", i.format)
#     print("  type", i.type)
#     print("  length", i.length)

In [56]:
# Dealing with multiple data files
# TODO: Make a def for doing this.
# Method1: Using pandas to read file.
# pdf_immigration = pd.read_sas(imm_data, format='sas7bdat', iterator=True, chunksize=5000000)
# pdf_immigration = pd.read_sas(imm_data, format='sas7bdat')

# imm_chunks_1 = list(pdf_immigration)


In [38]:
# from pandas.io.sas.sas7bdat import SAS7BDATReader

# rdr = SAS7BDATReader(imm_data, convert_header_text=False)
# df3 = rdr.read()

In [258]:
# TODO: Notification Table
"""
t2.imm_main_cic_id
t2.imm_per_cic_id
t2.news_cord_uid
src.cidemo_id
src.value_of_imm_destination_city
t2.news_title
t2.news_abstract
t2.news_publish_time
t2.news_authors
"""

#  ** t1: join imm two tables
#  ** t2: join news table with t1
#  ** t3: join us cities table with t2

df_notification = spark.sql(
    "WITH t1 AS \
    (SELECT * \
       FROM immigration_main_information_data imid \
      LEFT JOIN imm_personal ip \
             ON imid.imm_main_cic_id = ip.imm_per_cic_id \
        WHERE imid.imm_year = 2016 \
     ), t2 AS \
        (SELECT * \
           FROM t1 \
         LEFT JOIN news_article_data nad \
                ON t1.imm_arrival_date = nad.news_publish_time \
     )\
    SELECT * \
       FROM t2 \
     LEFT JOIN (SELECT * FROM us_cities_demographics_data ucdd INNER JOIN imm_destination_city_data idcd ON ucdd.cidemo_state_code = idcd.value_of_alias_imm_destination_city) src \
            ON t2.imm_port = src.code_of_imm_destination_city \
    "
).dropDuplicates()

df_notification.show(n=5, truncate=3)